In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt

In [2]:
evaluation =  pd.read_csv('C:\\Users\\Claudia\\Desktop\\1212\\sales_train_evaluation.csv',index_col=0)

In [3]:
item_names = list(evaluation['item_id'].unique())

In [4]:
calendar = pd.read_csv('C:\\Users\\Claudia\\Desktop\\1212\\calendar.csv',index_col=0)

In [5]:
# eva = pd.read_csv('C:\\Users\\Claudia\\Desktop\\1212\\sample_submission_eva.csv',index_col=0)

In [ ]:
def predict_all(item_list, train):
    items_predict = dict()
    for i in item_list:
        item = train[train['item_id']==i].iloc[:,3:].drop('state_id',axis=1)
        item.index = item['store_id']
        n = len(item['store_id'].unique())+1
        item=item.drop('store_id',axis=1)
        
        item_svd = SVD(n,item.T).T
        item_svd.index = pd.date_range('2011-01-29', periods=item_svd.shape[0], freq="D")
#         item_eva = eva.T.filter(regex=i)
#         item_eva.columns = item_svd.columns
        
        ema = EMA(item_svd.T)
#         sarima = SARIMA(item_svd)
        stl = STLF(item_svd)
        
        prediction = ((ema+stl)/2).round(0)
        items_predict[i] = prediction
    return items_predict

In [5]:
def SVD(n,df):
    U, sigma, VT = np.linalg.svd(np.array(df))
    k = n - sum(((sigma**2/sum(sigma**2)).cumsum()- 0.9) > 0)
    Uk = U[:, 0:k]
    Sk = np.diag(sigma[0:k])
    Vk = VT[0:k, :]
    TM = Uk.dot(Sk).dot(Vk)
    TM = pd.DataFrame(TM.T,columns = df.index, index = df.columns).round(2)
    TM[TM<=0] = 0.00001
    return TM

In [6]:
from statsmodels.tsa.api import ExponentialSmoothing

def EMA(df):
    ema = dict()
    for store in df.columns:
        EMA_fit = ExponentialSmoothing(df[store], seasonal_periods=180, trend='add', seasonal='add').fit(use_boxcox=True)
        fcast = EMA_fit.forecast(28)
        ema[store] = fcast
    ema=pd.DataFrame(ema)
    ema.index =  list(range(29))[1:]
    return ema

In [8]:
# # ***done
# import statsmodels.api as sm

# def SARIMA(df):
#     sarima = dict()
#     for store in df.columns:
#         SARIMA_fit = sm.tsa.statespace.SARIMAX(df[store], order=(0, 1, 0), seasonal_order=(2, 1, 2, 52)).fit()
#         fcast = SARIMA_fit.forecast(28).round(0)
#         sarima[store] = fcast
#     sarima=pd.DataFrame(sarima)
#     sarima.index =  list(range(29))[1:]
#     return sarima


In [6]:
from statsmodels.tsa.api import STLForecast
from statsmodels.tsa.arima.model import ARIMA

def STLF(df):
    stlf = dict()
    for store in df.columns:
#         stl_fit = STLForecast(df[store], ARIMA, model_kwargs={"order": (0, 1, 0)}).fit()
        stl_fit = STLForecast(df[store], ARIMA, model_kwargs={"order": (1, 1, 1)}, seasonal=365,).fit()
        forecasts = stl_fit.forecast(28)
        stlf[store] = forecasts.round(0)
    stlf=pd.DataFrame(stlf)
    stlf.index =  list(range(29))[1:]
    stlf[stlf<=0] = 0
    return stlf

In [ ]:
# sbmt for ewa / stl
import os
eDir = "C:\\Users\\Claudia\\Desktop\\1212\\stlf\\"
os.chdir(eDir)
allFiles = []
sbmt = pd.DataFrame()
i = 1
for eachfile in os.listdir():
    predict = pd.read_csv(eachfile,index_col=0)
    predict=predict.T.add_prefix('F')
    predict.index = eachfile.split('.')[0] + '_' + predict.index.astype(str) + '_evaluation'
    sbmt = sbmt.append(predict)
    
    v = pd.read_csv(eachfile,index_col=0)
    v=v.T.add_prefix('F')
    v.index = eachfile.split('.')[0] + '_' + v.index.astype(str) + '_validation'
    sbmt = sbmt.append(v)
    
    print(str(i)+'done')
    i += 1
#     if i == 3: break
    

sbmt.to_csv('C:\\Users\\Claudia\\Desktop\\1212\\stlf1212.csv')